In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import config
from torch.autograd import Variable

import pandas as pd

In [22]:
import pickle as plk

In [9]:
d = plk.load(open("zidian.txt", 'rb'))

In [10]:
train_info = pd.read_csv("train_info.csv")

In [11]:
filenames = train_info.filename

In [12]:
artists = train_info.artist

In [14]:
for i in range(len(train_info['artist'])):
    train_info['artist'][i] = d[train_info['artist'][i]]

/home/cai/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700


In [32]:
processed_train_info = pd.read_csv("processed_train_info.csv")

In [33]:
processed_train_info = processed_train_info.drop(['title', 'style', 'genre', 'date', 'Unnamed: 6', 'Unnamed: 0'], axis=1)

In [34]:
processed_train_info

,filename,artist
0,17354.jpg,68
1,95360.jpg,14
2,49757.jpg,25
3,53841.jpg,42
4,85979.jpg,59
5,81912.jpg,42
6,17146.jpg,72
7,46012.jpg,72
8,69300.jpg,331
9,5241.jpg,0


In [99]:
x = Variable(torch.rand(1, 3, 299, 299), requires_grad=False)

In [100]:
class shallowCNN(nn.Module):
    def __init__(self):
        #self.config = config
        super(shallowCNN, self).__init__()
        # [in, out, kernel_size, stride, padding]
        self.conv1 = nn.Conv2d(3, 16, 3, 1, 1)
        self.bn1 = nn.BatchNorm2d(16)
        self.max_pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)
        self.bn2 = nn.BatchNorm2d(32)
        self.max_pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(32, 64, 3, 1, 1)
        self.bn3 = nn.BatchNorm2d(64)
        self.max_pool3 = nn.MaxPool2d(2, 2)
        
        self.conv4 = nn.Conv2d(64, 128, 3, 1, 1)
        self.bn4 = nn.BatchNorm2d(128)
        self.max_pool4 = nn.MaxPool2d(2, 2)
        
        self.linear1 = nn.Linear(128 * 18 * 18, 2048)
        self.linear2 = nn.Linear(2048, 346)
    def forward(self, x):
        x = self.max_pool1(F.elu(self.bn1(self.conv1(x))))
        x = self.max_pool2(F.elu(self.bn2(self.conv2(x))))
        x = self.max_pool3(F.elu(self.bn3(self.conv3(x))))
        x = self.max_pool4(F.elu(self.bn4(self.conv4(x))))
        print(x.size())
        x = x.view(-1, x.size(1) * x.size(2) * x.size(3))
        x = F.elu(self.linear1(x))
        x = F.softmax(self.linear2(x))
        return x

In [101]:
model = shallowCNN()

In [102]:
y_hat = model(x)

torch.Size([1, 128, 18, 18])


In [66]:
loss_fn = nn.NLLoss()
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [42]:
xx = model(x).size()
xx.size()

RuntimeError: size mismatch, m1: [512 x 32], m2: [16384 x 2048] at /opt/conda/conda-bld/pytorch_1503970438496/work/torch/lib/TH/generic/THTensorMath.c:1293